In [2]:
from ase import Atoms
from ase.build import add_adsorbate, fcc100
from mace.calculators import mace_mp
from ase.build import surface
from ase.constraints import FixAtoms, UnitCellFilter
from ase.optimize import QuasiNewton
from ase.visualize import view
from ase.io import read, write
from ase.optimize import LBFGS
import numpy as np
from cu2o_bulk import cu2o_bulk, cu2o100, dimer1x1


bulk = cu2o_bulk()
bulk.calc = mace_mp(model="large", dispersion=True, default_dtype="float64", device='cuda')
ucf = UnitCellFilter(bulk)
LBFGS(ucf).run(fmax=0.01)
E_bulk = bulk.get_potential_energy()

n_layers=3
slab =dimer1x1(bulk, 3, 10)
slab.calc = mace_mp(model="large", dispersion=True, default_dtype="float64", device='cuda')

E_slab = slab.get_potential_energy()
E_cleav = (E_slab - E_bulk * n_layers) / 2 / np.linalg.det(slab.cell[:2, :2])
print(f'{n_layers=} {E_cleav=}')

bottom_Cu_z = np.min(slab[slab.symbols=='Cu'].positions[:,2])
mask1=slab.positions[:, 2] < bottom_Cu_z + 1.0
slab.set_constraint(FixAtoms(mask=mask1))

qn = LBFGS(slab, trajectory='Dimer100.traj')
write('Dimer100.traj', slab)
qn.run(fmax=0.01)
E_slab = slab.get_potential_energy()
#t = read('111slab.traj@:')
#view(t)


E_surf = (E_slab - E_bulk * n_layers) / 2 / np.linalg.det(slab.cell[:2, :2])
print(f'{n_layers=} {E_surf=}'

Using Materials Project MACE for MACECalculator with /home/lana/.cache/mace/46jrkm3v
Using float64 for MACECalculator, which is slower but more accurate. Recommended for geometry optimization.
       Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
LBFGS:    0 11:00:27      -28.924518*       0.0261
LBFGS:    1 11:00:27      -28.924546*       0.0243
LBFGS:    2 11:00:27      -28.924723*       0.0001
Using Materials Project MACE for MACECalculator with /home/lana/.cache/mace/46jrkm3v
Using float64 for MACECalculator, which is slower but more accurate. Recommended for geometry optimization.
n_layers=3 E_cleav=0.34113570029383317
       Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
LBFGS:    0 11:00:28      -74.254230*      41.0130
LBFGS:    1 11:00:28      -82.329872*       7.1377
LBFGS:    2 11:00:28      -82.870631*       4.6037
LBFGS:    3 11:00:28      -83.520894*       2.0294
LBFGS:    4 11:00:29

Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/lana/ve/lib64/python3.11/site-packages/ase/gui/pipe.py", line 32, in <module>
    main()
  File "/home/lana/ve/lib64/python3.11/site-packages/ase/gui/pipe.py", line 28, in main
    plt.show()
  File "/home/lana/ve/lib64/python3.11/site-packages/matplotlib/pyplot.py", line 607, in show
    return _get_backend_mod().show(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/lana/ve/lib64/python3.11/site-packages/matplotlib_inline/backend_inline.py", line 90, in show
    display(
  File "/home/lana/ve/lib64/python3.11/site-packages/IPython/core/display_functions.py", line 265, in display
    print(*objs)
ValueError: I/O operation on closed file.
